### Produce BioBERT embeddings for Drugs

#### Import libraries and BioBERT model

In [1]:
!pip install progressbar2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, TFBertModel
import pickle
import json
import numpy as np
from progressbar import ProgressBar

tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
model = TFBertModel.from_pretrained('dmis-lab/biobert-v1.1', from_pt=True)

C:\Users\dlezo\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


#### Load the Drugs common names from DrugBank

In [3]:
with open('./Data/TWOSIDES/rx_norm_to_name.pkl', 'rb') as file:
    rx_norm_to_name = pickle.load(file)

#### Get the embedding from BioBERT

In [4]:
embeddings = {}
i = 0
step = len(rx_norm_to_name.keys())//100
bar = ProgressBar(max_value=100)
for rx_norm_id in rx_norm_to_name.keys():
    if i%step==0:
        bar.next()
    encoded_input = tokenizer(rx_norm_to_name[rx_norm_id], return_tensors='tf')
    output_embedding = model(encoded_input)['last_hidden_state'][:,0,:]
    embeddings[rx_norm_id] = output_embedding
    i += 1
bar.finish()

100% (100 of 100) |######################| Elapsed Time: 0:08:20 Time:  0:08:200444


In [5]:
# Get the embeddings in a list
embeddings768 = []
for rx_norm_id in rx_norm_to_name.keys():
    embeddings768.append(embeddings[rx_norm_id].numpy()[0])
embeddings768 = np.array(embeddings768)

#### Reduce the embeddings dimensionality with PCA

In [9]:
import numpy as np
from sklearn.decomposition import PCA

size = 128
pca = PCA(n_components=size, svd_solver='full')
embeddings_pca = pca.fit_transform(embeddings768)

print('Number of components:', pca.n_components_)
print('Covariance covered:', pca.explained_variance_ratio_.sum())

Number of components: 128
Covariance covered: 0.9049438


In [10]:
embeddings_size_pca = {}
i = 0
for rx_norm_id in embeddings.keys():
    embeddings_size_pca[rx_norm_id] = embeddings_pca[i]
    i += 1

In [11]:
with open('./Data/Embeddings/biobert_embeddings_'+str(size)+'_pca.pkl', 'wb') as file:
    pickle.dump(embeddings_size_pca, file)
    print(file.name)


./Data/TWOSIDES-biobert_embeddings_128_pca.pkl
